# <center>Tarea 1 Extracción Automática de Información de la web y procesamiento de lenguaje natural</center>
### <center> Nicolás Carrillo Sepulveda - Ingenieria civil Telematica </center>
### <center> https://github.com/ncarrillo93/tarea1Scapping.git </center>
#### <center> 2021</center>

## <center>Pregunta 1</center>
#### Elegir un sitio web de noticias distinto a los tratados en clases (tambien distinto al de la/los compañera/os) y extraer, desde algun articulo de noticias en ese sitio web, el titulo y cuerpo de la noticia y, ademas, alguna variable de tipo numerica asociada al articulo. Como ejemplos de esto ultimo, se puede considerar: fecha de la noticia, cantidad de comentarios, cantidad de parrafos, cantidad de links asociados, etc.

In [96]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url="https://www.tarreo.com/noticias/620406/Scavlab-El-experimento-de-Scavengers-que-puede-tener-mas-de-9000-jugadores-al-mismo-tiempo"
html = urlopen(url)
bsObj = BeautifulSoup(html.read(),'html.parser')

#Titulo
titulo= bsObj.find('h1',attrs={'itemprop':'name headline'}).getText()
print('Titulo:'+str(titulo))

#Contenido
content = bsObj.find_all('div',attrs={'itemprop':'articleBody'})
for a in content:
    print( str(a.getText())+'\n \n' )

#Datos numericos ( Fecha,**cantidad de links asociados**)
    #fecha
time = bsObj.find('time').getText()
print('la hora en que se publico la noticia fue: '+str(time))
    #Cantidad de links asociados
links = bsObj.find_all('a',attrs={'target':'_blank'})
print(' la cantidad de link externo en la pagina son: '+str(len(links)) )

Titulo:Scavlab: El experimento de Scavengers que puede tener más de 9000 jugadores al mismo tiempo  



Editorial: Gaming / Facebook / Twitter / Cobertura / Instagram / Discord
Scavengers es un juego multijugador como muchos otros, su descripción oficial en Steam lo describe como un"«shooter» estratégico gratuito que mezcla PvE de mundo abierto y PvP de clases", pero hay algo que hace especial a Scavengers. El juego posee una modalidad multijugador como nunca se ha visto antes, podríamos decir que es un nuevo género dentro de los videojuegos multijugador, este modo experimental desarrollado por Midwinter Entertainment permite crear partidas capaces de albergar más 1700 usuarios en vivo y hasta 9000 jugadores de forma simultánea.
Recientemente James Davenport, un periodista del medio PC Gamer, tuvo la oportunidad de participar en una sesión de prueba ScavLab, una sesión en la que más de 1700 usuarios se unieron para participar del modo experimental del juego. Este modo en vez de realiza

## <center>Pregunta 2</center>
#### Expandir la extraccion automatizada de la pregunta anterior y programar un web crawler que 
* identifica donde esta la barra de busqueda en el sitio de noticias elegido, le añade un topico o tema particular 
* extrae desde todos los articulos encontrados en la primera pagina de resultados de busqueda,
    * todos los titulos
    * cuerpos de las noticias que se encuentran,
    * ademas de las variables numericas asociadas a cada articulo de noticia.

In [97]:
import requests
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

class Website:
    def __init__(self, name, url, searchUrl, resultListing, resultUrl, absoluteUrl):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl 
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl=absoluteUrl

class Articulo:
        def __init__(self,fecha,title,body,numParrafos):
            self.fecha = fecha
            self.title = title
            self.body = body
            self.numParrafos = numParrafos

class Crawler:

    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')  

    def getTitle(self,url): #OK
        try:
            html = urlopen(url)
        except HTTPError as e:
            return None
        try:
            bs = BeautifulSoup(html.read(),'lxml')
            aux= bs.find('title')
            for name in aux:
                titles = name
        except AttributeError as e:
            return None
        return titles
        
    def getBody(self,url):
        try:
            html = urlopen(url)
        except HTTPError as e:
            return None
        try:
            bs = BeautifulSoup(html.read(),'lxml')
            aux = bs.find('div',{'class':'main-single-body__content'})
            nameList = aux.findAll('p')
            noticias=' '
            for news in nameList:
                noticias= noticias + str(news.getText())
        except AttributeError as e:
            return None
        return noticias

    def getNumParrafos(self,url):
        try:
            html = urlopen(url)
        except HTTPError as e:
            return None
        try:
            bs = BeautifulSoup(html.read(),'lxml')
            aux = bs.find('div',{'class':'main-single-body__content'})
            nameList = aux.findAll('p')
        except AttributeError as e:
            return None
        return len(nameList)

    
    def getTime(self,url):
        try:
            html = urlopen(url)
        except HTTPError as e:
            return None
        try:
            bs = BeautifulSoup(html.read(),'lxml')
            time = bs.find('span',{'class':'main-single-about__item'}).getText()           
        except AttributeError as e:
            return None
        return time

    def getInfoArticulos(self,topic,site):

        #Obtiene la pagina
        bs = self.getPage(site.searchUrl + topic)
        #obtiene el div donde esta la información que quiero
        searchResults = bs.select(site.resultListing) #selecciona todos los elementos h1 de div.main-content (class="main-content")

        listArticulos =[]
        articulo = Articulo('Fecha','Titulos','Cuerpo de la noticia','Numero de parrafos')
        listArticulos.append(articulo)
        #recorre el arreglo para sacar la información de cada noticia
        for result in searchResults:
            #saca la pagina de la noticia en el buscador
            url = result.select(site.resultUrl)[0].attrs['href']
            if(site.absoluteUrl):
                bs = self.getPage(url)
            else:
                bs = self.getPage(site.url + url)
            
            if bs is None:
                print("El objeto beautifulSoup es None ")
                return
            title = self.getTitle( url )

            if title is None:
                print("noticia sin titulo o no se encontro")

            fecha = self.getTime(url)

            if fecha is None:
                print("noticia sin titulo o no se encontro")

            body  = self.getBody(url)
            numParrafos = 0
            if body is None:
                print("noticia sin cuerpo o no se encontro")
            else:
                i=0
                for name in body:
                    i = i+1
            numParrafos = i
            listArticulos.append(Articulo(self.getTime(url),self.getTitle(url),self.getBody(url),self.getNumParrafos(url)))
        return listArticulos


In [98]:
crawler = Crawler()
list = []
#                                          Website(name ,                url         ,           searchUrl              ,    resultListing    , resultUrl , absoluteUrl )
list = crawler.getInfoArticulos('Schalper',Website('cnn', 'https://www.cnnchile.com/','https://www.cnnchile.com/search/','div.main-content h2', 'div a'   , True        ))

## <center>Pregunta 3</center>
#### Finalmente, implementar una funcion de Python que permita guardar, en una base de datos ordenada y con formato CSV, la informacion extraida. Esto quiere decir que debe considerar guardar cada observacion (titulo del artuculo, cuerpo del articulo, variable numerica) como una fila del CSV.

In [99]:
import csv
#Funciona con la claseArticulo del ejercicio Pasado
class tablas:
    def crearCSV(self,list):
        csvFile=open('tabla.csv','wt+',encoding='utf-8')
        writer = csv.writer(csvFile)
        listAux=[]
        try:
            for i in list:
                if i.fecha is None:
                listAux=[i.fecha , i.title,i.body , i.numParrafos]
                writer.writerow(listAux)

        finally:
            csvFile.close()

#test
miTabla = tablas()
miTabla.crearCSV(list)
import pandas as pd 
df = pd.read_csv('tabla.csv',encoding='utf-8')